In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import cv2
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df=pd.read_csv('../input/plant-pathology-2020-fgvc7/train.csv')
test_df=pd.read_csv('../input/plant-pathology-2020-fgvc7/test.csv')
samp=pd.read_csv('../input/plant-pathology-2020-fgvc7/sample_submission.csv')

In [ ]:
train_df.head()

In [ ]:
samp.head()

In [ ]:
X_paths = [(os.path.join('../input/plant-pathology-2020-fgvc7/images',i+'.jpg')) for i in train_df.image_id]
train_df=train_df.drop(['image_id'],axis=1)
y_train = train_df.to_numpy().astype('float32')

In [ ]:
# plt.imshow(X_paths[0])
IMAGE_SIZE=100
images=[]
import os
for i in range(len(X_paths)):
    img = cv2.imread(os.path.join(X_paths[i]))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMAGE_SIZE,IMAGE_SIZE))
    images.append(img)#

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(images[0])
print(y_train[0])

In [ ]:
X_X = np.array(images).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)

In [ ]:
from keras.applications.vgg16 import VGG16
from keras import models
vgg= VGG16(include_top=False,pooling='avg',weights='imagenet',input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
# vgg.summary()

In [ ]:
from tensorflow.keras.layers import Dense,Dropout,Activation,MaxPooling2D,Flatten,Conv2D,BatchNormalization
from tensorflow.keras import Sequential
from tensorflow.keras import layers
import tensorflow as tf

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip(),
  layers.experimental.preprocessing.RandomRotation(0.5),
])

In [ ]:
#start building model....:::::
model=Sequential()
model.add(data_augmentation)

#layer1
model.add(vgg)

model.add(Dense(4))
model.add(BatchNormalization())
model.add(Activation('softmax'))
# model.layers[0].trainable=False

 # Freeze all the layers
for layer in vgg.layers[:-12]:
    layer.trainable = False
    
 # Check the trainable status of the individual layers
for layer in vgg.layers:
    print(layer, layer.trainable)
 
 
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
X = np.array(X_X)
Y=np.array(y_train)
modeL=model.fit(X,Y,batch_size=64,epochs=250,validation_split=0.3)

In [ ]:
plt.title('model accuracy')
plt.plot(modeL.history['val_accuracy'])
plt.plot(modeL.history['accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['val', 'train'], loc='upper left')
plt.show()

In [ ]:
X_test_paths = [(os.path.join('../input/plant-pathology-2020-fgvc7/images',i+'.jpg')) for i in test_df.image_id]

In [ ]:
X_test_paths
images_test=[]
import os
for i in range(len(X_test_paths)):
    img = cv2.imread(os.path.join(X_test_paths[i]))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMAGE_SIZE,IMAGE_SIZE))
    images_test.append(img)#

In [ ]:
X_T = np.array(images_test).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)

In [ ]:
y_pred = model.predict(np.array(X_T))

In [ ]:
temp=test_df.image_id


In [ ]:
final_sample=pd.DataFrame((temp))

In [ ]:
final_sample['healthy']=y_pred[:,0:1]
final_sample['multiple_diseases']=y_pred[:,1:2]
final_sample['rust']=y_pred[:,2:3]
final_sample['scab']=y_pred[:,3:4]

In [ ]:
final_sample.head()


In [ ]:
final_sample.to_csv('submission.csv',index=False)